#### Setup

In [18]:
## Basic imports and setup
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow.lite as tflite

from io import BytesIO
from urllib import request
from PIL import Image

In [2]:
model = keras.models.load_model('dino_dragon_10_0.899.h5')

#### Q1

In [4]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('dino_dragon_10_0.899.tflite', 'wb') as file:
    file.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\aaish\AppData\Local\Temp\tmpc8yzxli8\assets


INFO:tensorflow:Assets written to: C:\Users\aaish\AppData\Local\Temp\tmpc8yzxli8\assets


#### Q2

In [42]:
interpret = tflite.Interpreter(model_path='dino_dragon_10_0.899.tflite')
interpret.allocate_tensors()

in_index = interpret.get_input_details()[0]['index']
out_index = interpret.get_output_details()[0]['index']

out_index

13

#### Image prep

In [8]:
def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [13]:
img = download_image('https://upload.wikimedia.org/wikipedia/commons/thumb/d/df/Smaug_par_David_Demaret.jpg/1280px-Smaug_par_David_Demaret.jpg')
image_scaled = prepare_image(img, (150,150))

C:\Users\aaish\AppData\Local\Temp\ipykernel_102136\2564224172.py:12: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  img = img.resize(target_size, Image.NEAREST)


#### Q3

In [55]:
x = np.array(image_scaled, dtype='float32')
x = np.array([x])

x = x * 1./255

x[0][0][0]


array([0.5529412 , 0.31764707, 0.1764706 ], dtype=float32)

#### Q4

In [58]:
interpret.set_tensor(in_index, x)
interpret.invoke()
pred = interpret.get_tensor(out_index)
pred

array([[0.82448506]], dtype=float32)